# OmniSci Charting Example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) OmniSci charting example in Python with Altair, Vega Lite, and Vega:

![](https://cloud.githubusercontent.com/assets/2932405/25641647/1acce1f2-2f4a-11e7-87d4-a4e80cb262f5.gif)

First, import `altair`, `ibis`, and `ibis_vega_transform` and connect to the database:

In [2]:
import altair as alt
import ibis_vega_transform
import ibis.omniscidb


conn = ibis.omniscidb.connect(
    host='metis.mapd.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)

In [3]:
conn.list_tables()

['flights_donotmodify',
 'contributions_donotmodify',
 'tweets_nov_feb',
 'zipcodes_orig',
 'zipcodes',
 'demo_vote_clean',
 'us_faults',
 'zipcodes_2017',
 'us_county_level_tiger_edges_2018',
 'ca_roads_tiger',
 'input_node',
 'uk_wells',
 'RentalListings']

We can access the flights table:

In [4]:
t = conn.table("flights_donotmodify")

In [5]:
t

DatabaseTable[table]
  name: flights_donotmodify
  schema:
    flight_year : int16
    flight_month : int16
    flight_dayofmonth : int16
    flight_dayofweek : int16
    deptime : int16
    crsdeptime : int16
    arrtime : int16
    crsarrtime : int16
    uniquecarrier : string
    flightnum : int16
    tailnum : string
    actualelapsedtime : int16
    crselapsedtime : int16
    airtime : int16
    arrdelay : int16
    depdelay : int16
    origin : string
    dest : string
    distance : int16
    taxiin : int16
    taxiout : int16
    cancelled : int16
    cancellationcode : string
    diverted : int16
    carrierdelay : int16
    weatherdelay : int16
    nasdelay : int16
    securitydelay : int16
    lateaircraftdelay : int16
    dep_timestamp : timestamp
    arr_timestamp : timestamp
    carrier_name : string
    plane_type : string
    plane_manufacturer : string
    plane_issue_date : date
    plane_model : string
    plane_status : string
    plane_aircraft_type : string
    pl

It has 7 million rows:

In [6]:
t.info()

Table rows: 7009728

Column    Type      Non-null #
------    ----      ----------
flight_yearint16     7009728   
flight_monthint16     7009728   
flight_dayofmonthint16     7009728   
flight_dayofweekint16     7009728   
deptime   int16     6873482   
crsdeptimeint16     7009728   
arrtime   int16     6858079   
crsarrtimeint16     7009728   
uniquecarrierString(nullable=True)7009728   
flightnum int16     7009728   
tailnum   String(nullable=True)6926363   
actualelapsedtimeint16     6855029   
crselapsedtimeint16     7008884   
airtime   int16     6855029   
arrdelay  int16     6855029   
depdelay  int16     6873482   
origin    String(nullable=True)7009728   
dest      String(nullable=True)7009728   
distance  int16     7009728   
taxiin    int16     6858079   
taxiout   int16     6872670   
cancelled int16     7009728   
cancellationcodeString(nullable=True)137434    
diverted  int16     7009728   
carrierdelayint16     1524735   
weatherdelayint16     1524735   
nasdelay  int16 

In [12]:
t.head().execute()

,flight_year,flight_month,flight_dayofmonth,flight_dayofweek,deptime,crsdeptime,arrtime,crsarrtime,uniquecarrier,flightnum,...,dest_name,dest_city,dest_state,dest_country,dest_lat,dest_lon,origin_merc_x,origin_merc_y,dest_merc_x,dest_merc_y
0,2008,1,1,2,1922,1900,2046,2002,XE,2907,...,Lafayette Regional,Lafayette,LA,USA,30.205280,-91.987656,-10613169.0,3501040.00,-10240019.0,3529964.00
1,2008,1,18,5,1044,1015,1205,1139,XE,2545,...,Newark Intl,Newark,NJ,USA,40.692497,-74.168663,-7952336.5,5302010.00,-8256417.5,4967090.50
2,2008,1,5,6,1413,1405,1530,1524,XE,2155,...,George Bush Intercontinental,Houston,TX,USA,29.980473,-95.339722,-10027202.0,3804225.75,-10613169.0,3501040.00
3,2008,1,10,4,1544,1539,1650,1643,XE,2169,...,Monroe Regional,Monroe,LA,USA,32.510864,-92.037689,-10613169.0,3501040.00,-10245589.0,3830558.00
4,2008,1,6,7,1432,1435,1548,1554,XE,2849,...,Mobile Regional,Mobile,AL,USA,30.691416,-88.242836,-10613169.0,3501040.00,-9823147.0,3592738.25


## States

Now let's make the top left graph, the flights by state, letting you select one:

In [13]:
states = alt.selection_multi(fields=['origin_state'])

In [15]:
ibis_vega_transform.enable_debug()

In [17]:
flights_by_state = alt.Chart(
    t,
    title="Total Number of Flights by State"
).mark_bar().encode(
    x='count()',
    y=alt.Y(
        'origin_state',
        sort=alt.Sort(encoding='x', order='descending')
    ),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(
    states
)
flights_by_state

<IPython.core.display.JSON object>

alt.Chart(...)

Now let's add the little text there with the count as well.

In [18]:
ibis_vega_transform.disable_debug()

In [19]:
flights_by_state_text = alt.Chart(t).mark_text(dx=20).encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    text='count()'
)
flights_by_state_text

<IPython.core.display.JSON object>

alt.Chart(...)

We can combine them together now:

In [20]:
def create_flights_by_state(map_fn=lambda c: c):
    return map_fn(flights_by_state_text) + map_fn(flights_by_state)

In [21]:
create_flights_by_state()

alt.LayerChart(...)

## Delay chart

We do the same now for the delay chart:

In [22]:
airlines = alt.selection_multi(fields=['carrier_name'])

def create_carrier_delay(before_aggregate_fn=lambda c: c):
    carrier_delay = before_aggregate_fn(alt.Chart(
        t,
        title="Carrier Departure Delay by Arrival Delay (Minutes)"
    )).transform_aggregate(
        depdelay='mean(depdelay)',
        arrdelay='mean(arrdelay)',
        groupby=["carrier_name"]
    ).mark_point(filled=True, size=200).encode(
        x='depdelay',
        y='arrdelay',
        color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
        tooltip=['carrier_name', 'depdelay', 'arrdelay']
    ).add_selection(airlines)

    carrier_delay_text = before_aggregate_fn(alt.Chart(
        t
    )).transform_aggregate(
        depdelay='mean(depdelay)',
        arrdelay='mean(arrdelay)',
        groupby=["carrier_name"]
    ).mark_text().encode(
        x='depdelay',
        y='arrdelay',
        text='carrier_name',
    )
    return carrier_delay + carrier_delay_text

create_carrier_delay()

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

alt.LayerChart(...)

## States + Delays

Now we can combine these two charts, by adding filters by each others selections:

In [25]:
(
    create_flights_by_state(lambda c: c.transform_filter(airlines)) |
    create_carrier_delay(lambda c: c.transform_filter(states))
)

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

alt.HConcatChart(...)

In [27]:
ibis_vega_transform.disable_debug()

## Departure Time

Now we can add the final part, the depature time chart:

In [28]:
dates = alt.selection_interval(
    fields=['dep_timestamp'],
    encodings=['x'],
)


def create_time(map_fn=lambda c:c):
    return map_fn(
        alt.Chart(
            t,
            title='Number of Flights by Departure Time'
        )
    ).transform_filter(
        'datum.dep_timestamp != null'
    ).mark_line().encode(
        alt.X(
            'yearmonthdate(dep_timestamp):T',
        ),
        alt.Y(
            'count():Q',
            scale=alt.Scale(zero=False)
        )
    ).add_selection(
        dates
    )
create_time()

<IPython.core.display.JSON object>

alt.Chart(...)

## Combined

And add them all together!

In [29]:
HEIGHT = 800
WIDTH = 1000

(
        (
        create_flights_by_state(
            lambda c: c.transform_filter(
                {"and": [airlines, dates]}
            ).properties(
                height= 2 * HEIGHT / 3,
                width=WIDTH / 2
            )
        ) | 
        create_carrier_delay(
            lambda c: c.transform_filter(
                {"and": [dates, states]}
            )
        ).properties(
            height=2 * HEIGHT / 3,
            width=WIDTH / 2
        )
    ) & (
        create_time(
            lambda c: c.transform_filter(
                {"and": [airlines, states]}
            ).properties(
                height=HEIGHT / 3,
                width=WIDTH + 50
            )
        )
    )
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
).configure(
    autosize={
        'resize': True
    }
)

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

alt.VConcatChart(...)